### Validation and testing

In [ ]:
#@title Import libraries and dataset
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision import models
from torch import optim
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
#@title Define neural network, __init__ and forward functions

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
#@title Instantiate the model
net = Net()

In [ ]:
#@title Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#@title Load and transform the data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_transforms = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=(0.4914, 0.4822, 0.4465),
      std=(0.2023, 0.1994, 0.2010))])

train_data = CIFAR10(root="./train/",train=True,
                     download=True, transform=train_transforms)


train_set, val_set = random_split(train_data,[40000, 10000])

trainloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=16,
                    shuffle=True)

valloader = torch.utils.data.DataLoader(
                    val_set,
                    batch_size=16,
                    shuffle=True)

trainloader = torch.utils.data.DataLoader(train_set, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


100%|██████████| 170498071/170498071 [00:01<00:00, 91314826.56it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
#@title Set the model to training mode and evaluation mode for validation
for epoch in range(10):
    net.train()  # Set the model to training mode
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    net.eval()  # Set the model to evaluation mode for validation
    validation_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(trainloader)}, Validation Loss: {validation_loss / len(valloader)}, Validation Accuracy: {100 * correct / total}%')


Epoch 1, Training Loss: 1.8326236477464437, Validation Loss: 8.40141995973587, Validation Accuracy: 20.63%
Epoch 2, Training Loss: 1.5693841354757547, Validation Loss: 8.52663135919571, Validation Accuracy: 28.26%
Epoch 3, Training Loss: 1.482268398861587, Validation Loss: 8.750001503372193, Validation Accuracy: 24.43%
Epoch 4, Training Loss: 1.425206392236054, Validation Loss: 9.157256245040893, Validation Accuracy: 27.15%
Epoch 5, Training Loss: 1.3796403855096548, Validation Loss: 7.885831600093842, Validation Accuracy: 32.63%
Epoch 6, Training Loss: 1.3434773382876068, Validation Loss: 9.157978817272186, Validation Accuracy: 25.08%
Epoch 7, Training Loss: 1.320036523731053, Validation Loss: 9.06931521883011, Validation Accuracy: 27.09%
Epoch 8, Training Loss: 1.2966738528741524, Validation Loss: 8.871011086177825, Validation Accuracy: 29.03%
Epoch 9, Training Loss: 1.283492587737739, Validation Loss: 10.212591383552551, Validation Accuracy: 22.67%
Epoch 10, Training Loss: 1.2573963